In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
%%file sim_w_1d.f90

!>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

 subroutine SIM_W_1D(dt, km, rgas, gama, gm2, cp2, kappa, pe, dm2, pm2, pem, w2, dz2, pt2, ws, p_fac)
                        
   integer, intent(in):: km
   real,    intent(in):: dt, rgas, gama, kappa, p_fac
   real,    intent(in), dimension(km):: dm2, pt2, pm2, gm2, cp2
   real,    intent(in )::  ws
   real,    intent(in ), dimension(km+1):: pem
   real,    intent(out)::  pe(km+1)
   real,    intent(inout), dimension(km):: dz2, w2

! Local variables

   real, dimension(km  ):: Ak, Bk, Rk, Ck, w1, g_rat, bb, dd, aa, cc
   real, dimension(km+1):: pp, gam, wE, dm2e
   real  rdt, capa1, bet

   rdt   = 1. / dt
   capa1 = kappa - 1.

! Compute non-hydrostatic pert pressure

   do k = 1,km
    
     pe(k)   = exp(gm2(k)*log(-dm2(k)/dz2(k)*rgas*pt2(k))) - pm2(k)        
     w1(k)   = w2(k)
     dm2e(k) = dm2(k)
        
   enddo

   dm2e(km+1) = dm2(km)
    
! Set up tridiagonal coeffs for spline interpolation of w to grid edges. (Extra copy of bb for end calcs)

    do k = 1,km-1
       
      g_rat(k) = dm2(k)/dm2e(k+1)
    
      aa(k)    = 1.0
      bb(k)    = 2.*(1.+g_rat(k))
      cc(k)    = g_rat(k)
      dd(k)    = 3.*(w1(k) + cc(k)*w1(k+1))

    enddo
    
! Boundary conditions for wE = 0 at top

    bet    = bb(1)
    wE(1)  = 0.0
    wE(2)  = dd(1) / bet
    bb(km) = 2.
    dd(km) = 3.*w1(km)
       
! Forward calculation of tri-diagonal system

    do k=2,km
    
      gam(k)  =  g_rat(k-1) / bet
      bet     =  bb(k) - gam(k)
      wE(k+1) = (dd(k) - wE(k) ) / bet
         
    enddo

! Boundary conditions for wE = [ws(i) at ground]  
  
      wE(km+1) = 0.0

! Do the back substition, result is wE on zone edges.

    do k = km, 2, -1

      wE(k) = wE(k) - gam(k)*wE(k+1)
          
    enddo

! Compute cell centered tridiagonal coefficients 

    do k = 1, km
    
      aa(k) = dt * gm2(k) / dz2(k) * (pm2(k)+pe(k))
        
    enddo

! Compute edge centered tridiagonal coefficients and RHS

    do k = 2, km

      cc(k) = dt / (0.5 * (dm2(k) + dm2(k-1))) 
        
      Bk(k) = 1.0 - cc(k) * cc(k) + cc(k) * aa(k-1)
    
      Ak(k) = aa(k  )*cc(k)
        
      Ck(k) = aa(k-1)*cc(k)
    
      Rk(k) = wE(k) + cc(k) * (pe(k) - pe(k-1))

    enddo    

! Boundary value calc for forward tri-diagonal solution

   Rk(2)  = Rk(2) - 0.0 * Ck(2)      ! this includes the lower bc for w zero here

   bet    = Bk(2)
   wE( 2) = Rk(2) / bet
   Rk(km) = Rk(km) - 0.0 * Ck(km)   ! this includes the lower bc for w zero here
  
! Forward sweep.
    
    do k = 3, km

      gam(k) = Ck(k-1) / bet 
      bet    = Bk(k) - Ak(k) * gam(k)

      wE(k)  = ( Rk(k) - Ak(k) * wE(k-1) ) / bet

    enddo
    
    wE(km+1) = 0.0
        
! Back substitution for solution (wE = 0 at k=1)

    do k = km-1, 2, -1
      
      wE(k) = wE(k) - gam(k+1) * wE(k+1)
            
    enddo
    
! Solve for new perturbation pressure

    do k = 1, km

      pp(k)  = pp(k) - aa(k) * (wE(k+1) - wE(k))

    enddo
    
! Use new pp at cell centers to get new dz's

    do k = 1, km
    
      dz2(k) = -dm2(k)*rgas*pt2(k)*exp((cp2(k)-1.)*log(pp(k)+pm2(k)))

    enddo
    
! Need to generate new w at cell centers...

    w2(km) = ( wE(km+1) + 2.*wE(km) )*r3
    
    do k = km-1, 1, -1
      
      w2(k) = (wE(k) + bb(k)*wE(k+1) + g_rat(k)*wE(k+2))*r3 - g_rat(k)*w2(k+1)
          
    enddo
    
! Retrieve edge pressures

   pe(1) = 0.0

    do k = 1,km
      
      pe(k+1) = pe(k) + dm2(k)*(w2(k)-w1(k))*rdt
        
    enddo
    
 end subroutine SIM_W_1D

Overwriting sim_w_1d.f90


In [15]:
!fcompile.py



   ---> Removing all module files...safety first!


rm: cannot remove '*.mod': No such file or directory

Fortran object files compiled:   



rm: cannot remove 'sim_w_1d.so': No such file or directory


 ==> f2py --debug --fcompiler=gnu95 --f90flags='-O1 -funroll-loops -fopenmp -fPIC -I/usr/local/include' -DF2PY_REPORT_ON_ARRAY_COPY -c -m sim_w_1d sim_w_1d.f90  -lgomp -L/usr/local/lib  


running build
running config_cc
unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
build_src
building extension "sim_w_1d" sources
f2py options: []
f2py:> /tmp/tmplfgq_uqq/src.linux-x86_64-3.9/sim_w_1dmodule.c
creating /tmp/tmplfgq_uqq/src.linux-x86_64-3.9
Reading fortran codes...
	Reading file 'sim_w_1d.f90' (format:free)
Post-processing...
	Block: sim_w_1d
			Block: sim_w_1d
Post-processing (stage 2)...
Building modules...
	Building module "sim_